This notebook follows the same steps as the 2019 Split Data Cleanup. The full documentation can be found
over there.

In [1]:
import pandas as pd

from processing import cleaners, io, util

RACE_YEAR = 2022

## Split metadata

### Load raw metadata

In [2]:
df_split_info_raw = io.load_df_split_info_raw(RACE_YEAR)

### Cleanse the metadata

In [3]:
df_split_info = cleaners.process_df_split_info(df_split_info_raw)
df_split_info.drop(index='Kick to Finish 0.8 Miles Left', errors='ignore', inplace=True)
df_split_info['cutoff_hr'] = [3.75, 6., 7.5, 9.5, 12., 14., None, 18., 21.25, 23., 26.5, 30.]

Verify that the split info didn't change from year to year.

In [4]:
df_split_info_2019 = io.load_df_split_info_clean(2019)
pd.testing.assert_series_equal(df_split_info['distance_mi'], df_split_info_2019['distance_mi'])


### Save cleaned metadata

In [5]:
io.save_df_split_info_clean(df_split_info, RACE_YEAR)

## Athlete split data

### Load data

In [6]:
df_split_times_raw = io.load_df_split_times_raw(RACE_YEAR)

### Cleanse data

In [7]:
df_split_times = cleaners.sort_df_split_data(df_split_times_raw, df_split_info)

util.display_full_df(util.df_td_fmt(df_split_times))

,Adrian Macdonald,JP Giblin,Ryan Kaiser,Tyler Andrews,Christopher Harrington,Devon Colegrove,Joel Anderson,Joshua Eberly,BRIAN M BABYAK,Guillaume Perrot,Marvin Sandoval,Clare Gallagher,Todd Dill,Sean Bartlett,Matthew Vira,Devin Vennard,Fan Zhou,Duncan Callahan,Rodrigo Jimenez,Mickey Davis,Michael Martinez,Martin Malherbe,Patrick Cade,Zachary Russell,Fabio Di Giacomo,Cory Logsdon,Alisyn Hummelberg,Chad Lasater,Max Robinson,Daniel Harper,Ryan Simmons,Ted Schultz,Wesley Sandoval,Ryan Guldan,Francesco Perini,Ryan Van Duzer,Leonardo Santillana,Lindsey Herman,Brian Husmann,Eric Holmstead,Chris Wright,Benjamin Boese,Robert Strickland,Patrick McClellan,Ryan Charter,Roxanne Vogel,Tim Hoppin,Jason Clarke,Eric Pepper,COREY KALLEMBACH,Max Kilcoyne,Matthew Berdine,Adam Wolf,Noah Schaich,Madeline Hirschfeld,Bruce Dailey,Michael Nanaszko,Paul Steinweg,Brian Turner,Peter Hogg,Brian Klug,Rachel Brewer,Kris Rugloski,Samantha Wood,Max Riehemann,Neil Feldman,Kevin Carson,Piers Constable,Drew Petersen,Larry Smith,Kirill Minkovich,Jonathan Clinthorne,David Miao,Peter Polito,Jacob Sheetz-Willard,Shelby Johnson,BRIAN TOY,Michael New,Don Reichelt,David Duede,Bedrich Stancik,Rob Kelsey,John Pedrosa,Chris Kelly,Darian Allberry,Kevin Linebarger,Aaron Anton,MAX FULTON,April McAnly,Nathan Sicher,Robert Ankarlo,Pierce Showe,Martin Almazan,Mathieu Shields,Abe Kane,Kendall Dinwiddie,Jolene Sandoval,Jared Sweet,Dustin Kessinger,Andrew Elvester,Kay Borleis,Elizabeth Helland,Megan Eckert,Kayde Anderson,Kate LeVasseur,Jordan Akiyoshi,Trevor Heller,Ash Walsh,Katy Anderson,Andrew Stanley,Alexander Kellam,Brooks Williams,Emily Luhrs,Perry Curiston,Michael Oliva,Gergely Medgyessy,Kevin Bliss,Erik Brandner,Gwen Golaszewski,SHINSUKE ISOMURA,Paul Prawdzik,Henry Fischer,Alex Falconer,Nate Glocke,Tim Cronk,Dani Reese,Noah Sosin,Ivan O' Gorman,Sanami Fottral,Kris Winslett,Patrick Atkinson,Todd Stiefler,Nick Harreld,Eugene Betty,Thomas McPhaul,Lisa Albers,John Diana,Sierra DeGroff,Jonathan Norloff,Michael Sager,Scott Marten,William Rushforth,Brad Slighting,Megan Flanagan,Gary Geer,Michael Curry,Jenna Carosio,Hellah Sidibe,Tatiana Rypinski,Samantha Hurlbutt,Maik Becker,Josh Gall,Peter Mack,Paul Schoenlaub,Kara Diamond-Husmann,John Stanley,DANIEL PRITCHARD,TIMOTHY BARR,Miguel Perez,Simon Guerard,Christopher Minton,Nick Smarrelli,Collin Morris,Christopher Nicholas,Dolan Potts,Jeremy Segal,Tess Hamilton,Bram Blondelle,Brent Harder,Tommy Padia,Nicholas Mullen,Heather Kopsky,Karl Huchthausen,Matthew Burdette,Todd Meconis,Breanna Cornell,Abraham Bejil,David Long,Diego Garcia,Andrew Catalano,Marcelo Arias Massa,Todd Pizzi,Jan Olsson,Chris Lantzy,Phil Atkinson,Travis Tung,John Molloy,John Minkema,Junko Kazukawa,Gary Lampman,Matthew Tarver,Kristin Gablehouse,Jason Friedman,Jonathan Prosser,Beau Brooks,Sam Schwaller,Mason McWilliams,Sarah Hodges,Brad Stahlman,Sergio Saenz,Joel Beskar,Peter Keirn,Jerry Scrogham,Greg Newman,Neil Smith,Andrew Mueller,Celesta Cairns,Peter Hall,Reid Longabaugh,Carrie Stafford,Christian Robbins,Adam Trubow,KEvin McCabe,Robert Perednia,Doug Gruenbacher,Charles Salsgiver,Andrea Clemons,Charlie Ealy,Oleksii Kononenko,Erin MacNabb,Nicholas Kierstead,Sean Humphrey,Jeff Rowthorn,Jeremy Couch,Mike Brown,Tyler Hall,Mike Peter,Sally McRae,Jeff Hirvela,Megan Frisk,Matthew Young,Harvey Lewis,Antonio Pizano,Scott Davis,Gideon Scoggin,Daniel Polski,Elisha Tilton,Bryce Weeter,Benjamin Jones,Brian Smith,Mike Van Ryswyck,John Stubbs,JESSE ALEXANDER,Lisa Hallett,Kate Laing,Heather Griffith,Craig Bachrodt,Bonnie Alexander,Julian Tonsmeire,Jared Bruggeman,Brian Passenti,Bruce Commander,Samuel Kane,John Wiencek,Jason Williams,David Lee,Patrick Sodia,Kyle Rodemacher,Amy Clark,Brent Bozarth,Tim Corliss,Paul Nielsen,Jonathan Puskas,Zach Jones,Andrew Magalong,William Stoval,John Peyrebrune,Michael Zettlemoyer,Alan Villavicencio,Gunnar Rogers,Michael Chavez,Clarence Marvan,Mark Gaffney,Mandy Mullen,Dan Williams,Bradley MacHamer,Maria Simone,Matthew Pate,Julian Serrano,Jeremey Frye,

#### Remove bad data

First up, we have a couple of people who registered a finish line split by accident:

In [8]:
util.df_td_fmt(df_split_times.loc[:, ['Mickey Davis', 'Sean Bartlett']])

,Mickey Davis,Sean Bartlett
label,,
12.6mi_Start to May Queen,02:01:26,<NA>
23.5mi_Start to Outward Bound OUT,04:04:24,<NA>
29.3mi_Start to Half Pipe Out,04:55:04,<NA>
37.9mi_Start to Twin Lakes Out,06:46:29,<NA>
43.5mi_Start to Hope Pass Out,08:48:50,<NA>
50mi_Start to Winfield,10:37:12,<NA>
56.5mi_Start to Hope Pass In,12:57:53,<NA>
62.5mi_Start to Twin Lakes In,15:35:06,<NA>
71.1mi_Start to Half Pipe In,19:56:20,<NA>


These errors can have a real impact: Mickey Davis would have been the third overall female if
her finish line split were valid. The LT100 social media team even tweeted about it before the official
awards ceremony!

Let's delete these invalid split times:

In [9]:
df_split_times.loc['Full Course', 'Mickey Davis'] = pd.NA
df_split_times.loc['Full Course', 'Sean Bartlett'] = pd.NA

Like a lot of the faster runners this year, eventual winner Adrian Macdonald was having some trouble with
his split tracking.

In [10]:
util.series_td_fmt(df_split_times['Adrian Macdonald'])

label
12.6mi_Start to May Queen                <NA>
23.5mi_Start to Outward Bound OUT        <NA>
29.3mi_Start to Half Pipe Out        04:57:21
37.9mi_Start to Twin Lakes Out       05:17:05
43.5mi_Start to Hope Pass Out        06:43:22
50mi_Start to Winfield               07:40:30
56.5mi_Start to Hope Pass In         09:09:30
62.5mi_Start to Twin Lakes In        09:59:11
71.1mi_Start to Half Pipe In         11:30:50
76.9mi_Start to Outward Bound In     12:08:01
87.8mi_Start to May Queen In         14:06:25
Full Course                          16:05:44
Name: Adrian Macdonald, dtype: string

If that split at mile 29.3 were accurate, he would have covered the next 8.6 miles in under 20 minutes.
By comparing his data to his closest competitors, it's clear that his first recorded split was a blooper.

And that's a bummer, because he ran one of the fastest races in history, and his data would contribute
to some valuable insights about strategy! Stay tuned, because I might end up pouring over social media
or Strava to tease out that info.

In [11]:
df_split_times.loc['29.3mi_Start to Half Pipe Out', 'Adrian Macdonald'] = pd.NA

Next is an anomalous split that only became apparent when I was plotting the data at a later step.

In [12]:
util.df_td_fmt(
  util.get_column_context(df_split_times, 'Timothy Weng')
)

,Craig Sylvester,Josh Tavarez,Hawaiian Shirt Ray Churgovich,Andrew JOHNSON,Cecelia Williams,Timothy Weng,Margaret Spring,Katrin Silva,Ryan Mcafee,Matthew Johnson
label,,,,,,,,,,
12.6mi_Start to May Queen,02:22:38,02:42:12,02:06:28,01:56:07,02:26:23,02:31:50,02:15:00,02:29:22,02:24:44,02:05:22
23.5mi_Start to Outward Bound OUT,04:50:11,<NA>,04:29:00,03:54:01,04:31:46,04:56:42,04:25:08,04:51:20,04:52:02,04:21:54
29.3mi_Start to Half Pipe Out,05:55:11,05:46:15,05:30:07,04:42:36,05:20:07,06:03:19,05:20:03,05:46:02,05:47:11,05:11:09
37.9mi_Start to Twin Lakes Out,08:24:41,08:02:31,07:47:57,06:34:20,07:18:18,08:34:06,07:36:49,07:58:05,07:59:41,07:11:42
43.5mi_Start to Hope Pass Out,11:00:20,10:17:50,10:19:50,08:39:24,09:27:02,11:38:37,09:46:35,10:15:43,10:30:27,09:26:39
50mi_Start to Winfield,13:02:57,11:59:35,12:05:42,10:19:24,11:01:05,13:36:16,11:27:24,11:57:02,12:24:07,11:16:34
56.5mi_Start to Hope Pass In,15:28:16,14:35:02,14:59:18,12:27:21,13:15:25,16:24:36,13:36:20,<NA>,14:57:01,13:45:32
62.5mi_Start to Twin Lakes In,17:40:12,<NA>,17:15:56,14:21:25,14:57:56,<NA>,15:19:37,16:29:11,16:59:54,15:46:01
71.1mi_Start to Half Pipe In,21:15:18,19:50:44,20:59:59,16:51:10,17:45:51,<NA>,17:50:48,19:16:12,20:11:42,19:50:44


Looks like Timothy Weng DNFed at mile 56.5, but recorded an accidental split at mile 76.9.
The athletes who came through Outward Bound (his last recorded split) with a similar time
had all been running much faster.

In [13]:
df_split_times.loc['76.9mi_Start to Outward Bound In', 'Timothy Weng'] = pd.NA

Finally, there is one athlete that seems to have some bad data, but it's not clear exactly where.

In [14]:
util.df_td_fmt(
  util.get_column_context(df_split_times, 'Michael Nanaszko')
)

,Matthew Berdine,Adam Wolf,Noah Schaich,Madeline Hirschfeld,Bruce Dailey,Michael Nanaszko,Paul Steinweg,Brian Turner,Peter Hogg,Brian Klug
label,,,,,,,,,,
12.6mi_Start to May Queen,02:09:24,02:09:18,02:01:40,01:54:29,02:05:45,<NA>,02:11:37,02:10:48,02:09:54,02:21:11
23.5mi_Start to Outward Bound OUT,04:12:17,04:17:38,03:52:28,04:32:40,04:14:32,<NA>,04:09:46,04:13:43,04:16:42,<NA>
29.3mi_Start to Half Pipe Out,04:58:55,05:06:27,04:33:46,05:26:06,05:03:51,<NA>,04:56:31,05:02:36,05:02:37,05:11:04
37.9mi_Start to Twin Lakes Out,06:59:21,07:15:24,06:17:42,07:25:11,07:01:28,05:10:20,06:43:59,07:02:06,07:02:05,07:05:45
43.5mi_Start to Hope Pass Out,08:58:16,09:18:05,08:07:56,09:26:21,09:00:43,07:43:45,08:39:20,09:10:03,09:10:02,09:16:10
50mi_Start to Winfield,10:25:35,10:44:04,09:25:25,10:57:04,10:32:38,08:56:05,10:07:50,10:46:16,10:46:17,10:55:45
56.5mi_Start to Hope Pass In,12:35:12,<NA>,11:17:50,12:49:48,12:42:06,11:11:14,12:22:27,13:02:32,13:02:36,<NA>
62.5mi_Start to Twin Lakes In,13:58:51,14:12:53,12:49:16,14:11:18,14:15:27,<NA>,14:00:38,14:40:40,14:40:40,14:47:48
71.1mi_Start to Half Pipe In,16:30:26,16:27:16,<NA>,16:31:54,16:43:40,<NA>,<NA>,17:02:37,17:02:52,<NA>


His first recorded split (5:10:20 to Twin Lakes) would put him in first at that point.
But his next two splits are pretty fast for his eventual finish time too.
It could have been a wonky chip, but I cannot rule out that he went out aggressively fast
and paid the price later.

I am tempted to just throw out all his splits because something seems wrong. Leaving them
for now though.

Now to look for negative elapsed times:

In [15]:
util.df_td_fmt(
  util.get_df_negative_timedelta(df_split_times)
)

,Kris Rugloski
label,
12.6mi_Start to May Queen,<NA>
23.5mi_Start to Outward Bound OUT,04:41:30
29.3mi_Start to Half Pipe Out,<NA>
37.9mi_Start to Twin Lakes Out,07:41:59
43.5mi_Start to Hope Pass Out,13:32:49
50mi_Start to Winfield,11:20:05
56.5mi_Start to Hope Pass In,<NA>
62.5mi_Start to Twin Lakes In,14:43:36
71.1mi_Start to Half Pipe In,17:17:24


In [16]:
util.df_td_fmt(
  util.get_column_context(df_split_times, 'Kris Rugloski')
)

,Paul Steinweg,Brian Turner,Peter Hogg,Brian Klug,Rachel Brewer,Kris Rugloski,Samantha Wood,Max Riehemann,Neil Feldman,Kevin Carson
label,,,,,,,,,,
12.6mi_Start to May Queen,02:11:37,02:10:48,02:09:54,02:21:11,02:14:29,<NA>,02:20:48,02:09:20,02:25:35,01:56:18
23.5mi_Start to Outward Bound OUT,04:09:46,04:13:43,04:16:42,<NA>,04:25:13,04:41:30,04:27:51,04:04:15,04:48:22,03:47:05
29.3mi_Start to Half Pipe Out,04:56:31,05:02:36,05:02:37,05:11:04,05:12:49,<NA>,05:15:07,04:54:00,05:44:21,04:30:18
37.9mi_Start to Twin Lakes Out,06:43:59,07:02:06,07:02:05,07:05:45,07:10:57,07:41:59,07:08:44,06:38:23,07:48:56,06:20:05
43.5mi_Start to Hope Pass Out,08:39:20,09:10:03,09:10:02,09:16:10,09:12:10,13:32:49,09:16:47,08:46:19,09:43:14,08:14:46
50mi_Start to Winfield,10:07:50,10:46:16,10:46:17,10:55:45,10:45:53,11:20:05,10:39:13,10:24:01,11:14:16,09:34:22
56.5mi_Start to Hope Pass In,12:22:27,13:02:32,13:02:36,<NA>,12:57:12,<NA>,12:37:45,12:32:45,13:21:01,11:43:57
62.5mi_Start to Twin Lakes In,14:00:38,14:40:40,14:40:40,14:47:48,14:25:56,14:43:36,14:03:17,13:51:36,14:41:14,13:30:52
71.1mi_Start to Half Pipe In,<NA>,17:02:37,17:02:52,<NA>,16:57:50,17:17:24,16:27:18,16:14:46,17:16:28,15:55:39


It looks like Kris Rugloski's chip didn't fire at the Hope Pass station on the way out, but it
did on the way in. Let's switch 'em.

In [17]:
df_split_times.loc['56.5mi_Start to Hope Pass In', 'Kris Rugloski'] = df_split_times.loc[
                   '43.5mi_Start to Hope Pass Out', 'Kris Rugloski']
df_split_times.loc['43.5mi_Start to Hope Pass Out', 'Kris Rugloski'] = pd.NA

In [18]:
df_split_times = cleaners.sort_df_split_data(df_split_times, df_split_info)

util.display_full_df(util.df_td_fmt(df_split_times))

,Adrian Macdonald,JP Giblin,Ryan Kaiser,Tyler Andrews,Christopher Harrington,Devon Colegrove,Joel Anderson,Joshua Eberly,BRIAN M BABYAK,Guillaume Perrot,Marvin Sandoval,Clare Gallagher,Todd Dill,Matthew Vira,Devin Vennard,Fan Zhou,Duncan Callahan,Rodrigo Jimenez,Michael Martinez,Martin Malherbe,Patrick Cade,Zachary Russell,Fabio Di Giacomo,Cory Logsdon,Alisyn Hummelberg,Chad Lasater,Max Robinson,Daniel Harper,Ryan Simmons,Ted Schultz,Wesley Sandoval,Ryan Guldan,Francesco Perini,Ryan Van Duzer,Leonardo Santillana,Lindsey Herman,Brian Husmann,Eric Holmstead,Chris Wright,Benjamin Boese,Robert Strickland,Patrick McClellan,Ryan Charter,Roxanne Vogel,Tim Hoppin,Jason Clarke,Eric Pepper,COREY KALLEMBACH,Max Kilcoyne,Matthew Berdine,Adam Wolf,Noah Schaich,Madeline Hirschfeld,Bruce Dailey,Michael Nanaszko,Paul Steinweg,Brian Turner,Peter Hogg,Brian Klug,Rachel Brewer,Kris Rugloski,Samantha Wood,Max Riehemann,Neil Feldman,Kevin Carson,Piers Constable,Drew Petersen,Larry Smith,Kirill Minkovich,Jonathan Clinthorne,David Miao,Peter Polito,Jacob Sheetz-Willard,Shelby Johnson,BRIAN TOY,Michael New,Don Reichelt,David Duede,Bedrich Stancik,Rob Kelsey,John Pedrosa,Chris Kelly,Darian Allberry,Kevin Linebarger,Aaron Anton,MAX FULTON,April McAnly,Nathan Sicher,Robert Ankarlo,Pierce Showe,Martin Almazan,Mathieu Shields,Abe Kane,Kendall Dinwiddie,Jolene Sandoval,Jared Sweet,Dustin Kessinger,Andrew Elvester,Kay Borleis,Elizabeth Helland,Megan Eckert,Kayde Anderson,Kate LeVasseur,Jordan Akiyoshi,Trevor Heller,Ash Walsh,Katy Anderson,Andrew Stanley,Alexander Kellam,Brooks Williams,Emily Luhrs,Perry Curiston,Michael Oliva,Gergely Medgyessy,Kevin Bliss,Erik Brandner,Gwen Golaszewski,SHINSUKE ISOMURA,Paul Prawdzik,Henry Fischer,Alex Falconer,Nate Glocke,Tim Cronk,Dani Reese,Noah Sosin,Ivan O' Gorman,Sanami Fottral,Kris Winslett,Patrick Atkinson,Todd Stiefler,Nick Harreld,Eugene Betty,Thomas McPhaul,Lisa Albers,John Diana,Sierra DeGroff,Jonathan Norloff,Michael Sager,Scott Marten,William Rushforth,Brad Slighting,Megan Flanagan,Gary Geer,Michael Curry,Jenna Carosio,Hellah Sidibe,Tatiana Rypinski,Samantha Hurlbutt,Maik Becker,Josh Gall,Peter Mack,Paul Schoenlaub,Kara Diamond-Husmann,John Stanley,DANIEL PRITCHARD,TIMOTHY BARR,Miguel Perez,Simon Guerard,Christopher Minton,Nick Smarrelli,Collin Morris,Christopher Nicholas,Dolan Potts,Jeremy Segal,Tess Hamilton,Bram Blondelle,Brent Harder,Tommy Padia,Nicholas Mullen,Heather Kopsky,Karl Huchthausen,Matthew Burdette,Todd Meconis,Breanna Cornell,Abraham Bejil,David Long,Diego Garcia,Andrew Catalano,Marcelo Arias Massa,Todd Pizzi,Jan Olsson,Chris Lantzy,Phil Atkinson,Travis Tung,John Molloy,John Minkema,Junko Kazukawa,Gary Lampman,Matthew Tarver,Kristin Gablehouse,Jason Friedman,Jonathan Prosser,Beau Brooks,Sam Schwaller,Mason McWilliams,Sarah Hodges,Brad Stahlman,Sergio Saenz,Joel Beskar,Peter Keirn,Jerry Scrogham,Greg Newman,Neil Smith,Andrew Mueller,Celesta Cairns,Peter Hall,Reid Longabaugh,Carrie Stafford,Christian Robbins,Adam Trubow,KEvin McCabe,Robert Perednia,Doug Gruenbacher,Charles Salsgiver,Andrea Clemons,Charlie Ealy,Oleksii Kononenko,Erin MacNabb,Nicholas Kierstead,Sean Humphrey,Jeff Rowthorn,Jeremy Couch,Mike Brown,Tyler Hall,Mike Peter,Sally McRae,Jeff Hirvela,Megan Frisk,Matthew Young,Harvey Lewis,Antonio Pizano,Scott Davis,Gideon Scoggin,Daniel Polski,Elisha Tilton,Bryce Weeter,Benjamin Jones,Brian Smith,Mike Van Ryswyck,John Stubbs,JESSE ALEXANDER,Lisa Hallett,Kate Laing,Heather Griffith,Craig Bachrodt,Bonnie Alexander,Julian Tonsmeire,Jared Bruggeman,Brian Passenti,Bruce Commander,Samuel Kane,John Wiencek,Jason Williams,David Lee,Patrick Sodia,Kyle Rodemacher,Amy Clark,Brent Bozarth,Tim Corliss,Paul Nielsen,Jonathan Puskas,Zach Jones,Andrew Magalong,William Stoval,John Peyrebrune,Michael Zettlemoyer,Alan Villavicencio,Gunnar Rogers,Michael Chavez,Clarence Marvan,Mark Gaffney,Mandy Mullen,Dan Williams,Bradley MacHamer,Maria Simone,Matthew Pate,Julian Serrano,Jeremey Frye,Isaac Mousel,Scott Rubinste

### Save the cleaned data

In [19]:
io.save_df_split_times_clean(df_split_times, RACE_YEAR)